In [53]:
import pandas as pd

pd.options.plotting.backend = "plotly"
import plotly
import plotly.graph_objects as go


import plotly.io as pio
import plotly.express as px

pio.templates.default = "simple_white"

In [2]:
time_taken = pd.Series([1174.0,
573.0,
358.0,
983.0,
455.0,
242.0,
255.0,
389.0,
323.0,
465.0,
436.0,
1094.0,
283.0,
947.0,
291.0,
719.0,
364.0,
343.0,
185.0,
271.0,
685.0,
342.0,
1752.0,
215.0,
2324.0,
747.0,
571.0,
477.0,
654.0,
288.0,
419.0,
235.0,
347.0,
785.0,
678.0,
716.0,
474.0,
334.0,
891.0,
268.0,
271.0,
652.0,
344.0,
170.0,
249.0,
225.0,
349.0,
231.0,
376.0,
486.0,
875.0,
298.0,
269.0,
801.0,
295.0,
635.0,
370.0,
145.0,
698.0,
409.0,
678.0,
282.0,
305.0,
372.0,
465.0,
639.0,
202.0,
285.0,
627.0,
364.0,
792.0,
616.0,
398.0,
473.0,
647.0,
332.0,
1656.0,
202.0,
279.0,
835.0,
678.0,
157.0,
216.0,
693.0,
261.0,
241.0,
275.0,
542.0,
456.0,
383.0,
446.0,
377.0,
332.0,
399.0,
309.0,
585.0,
670.0,
420.0]
)

In [3]:
time_taken.plot.hist()

In [4]:
mean_std_series = pd.Series(
    [(x - time_taken.mean()) / time_taken.std() for x in time_taken.to_list()]
)
mean_std_series.loc[
    mean_std_series < mean_std_series.mean() - 3 * mean_std_series.std()
]

Series([], dtype: float64)

In [5]:
mean_std_series.hist()

In [12]:
data_usa = pd.read_csv("../data/raw/usa/all_apps_wide_2023-11-14_USA.csv")
data_deu_1 = pd.read_csv("../data/raw/deutsch/all_apps_wide_2023-11-14_DEU.csv")
data_deu_2 = pd.read_csv(
    "../data/raw/deutsch runde 2/all_apps_wide_2023-11-15_DEU.csv"
)

In [35]:
relevant_columns = [
    "survey.1.player.positive_szenarios",
    "survey.1.player.treatment_pa",
    "survey.1.player.PRS1T1",
    "survey.1.player.PRS1T2",
    "survey.1.player.PRS2T1",
    "survey.1.player.PRS2T2",
    "survey.1.player.PRS3T1",
    "survey.1.player.PRS3T2",
    "survey.1.player.PRS4T1",
    "survey.1.player.PRS4T2",
    "survey.1.player.FRS1T1",
    "survey.1.player.FRS1T2",
    "survey.1.player.FRS2T1",
    "survey.1.player.FRS2T2",
    "survey.1.player.FRS3T1",
    "survey.1.player.FRS3T2",
    "survey.1.player.FRS4T1",
    "survey.1.player.FRS4T2",
    "survey.1.player.ARS1T1",
    "survey.1.player.ARS1T2",
    "survey.1.player.ARS2T1",
    "survey.1.player.ARS2T2",
    "survey.1.player.ARS3T1",
    "survey.1.player.ARS3T2",
    "survey.1.player.ARS4T1",
    "survey.1.player.ARS4T2",
    "survey.1.player.ETHS1T1",
    "survey.1.player.ETHS1T2",
    "survey.1.player.ETHS2T1",
    "survey.1.player.ETHS2T2",
    "survey.1.player.ETHS3T1",
    "survey.1.player.ETHS3T2",
    "survey.1.player.ETHS4T1",
    "survey.1.player.ETHS4T2",
    "survey.1.player.PROS1T1",
    "survey.1.player.PROS1T2",
    "survey.1.player.PROS2T1",
    "survey.1.player.PROS2T2",
    "survey.1.player.PROS3T1",
    "survey.1.player.PROS3T2",
    "survey.1.player.PROS4T1",
    "survey.1.player.PROS4T2",
    "origin"
]

In [39]:
data_all = (
    pd.concat(
    [
        data_usa.assign(origin="USA"),
        data_deu_1.assign(origin="DEU"),
        data_deu_2.assign(origin="DEU"),
    ],
    axis=0,
    )
    .loc[lambda df_: df_["participant._current_page_name"] == "End"]
    .loc[:, relevant_columns]
    .rename(columns=dict(zip(relevant_columns, [x.replace("survey.1.player.","") for x in relevant_columns])))
)

In [ ]:
# FRS1T1 feel responsible senario 1, team lead 1 (familiar solution)
# ARS1T1 act responsible senario 1, team lead 1 (familiar solution)

# FRS1T1 feel responsible senario 1, team lead 1 (new solution)
# ARS1T1 act responsible senario 1, team lead 1 (new solution)

In [55]:
data_all.groupby(["positive_szenarios", "treatment_pa"])["FRS1T2"].mean()

positive_szenarios  treatment_pa
1_and_4             1_and_4         6.255319
                    2_and_3         6.200000
2_and_3             1_and_4         6.052632
                    2_and_3         5.800000
Name: FRS1T2, dtype: float64

In [57]:
FRS1T1_df

,outcome,pa,FRS1T1
0,positive,yes,4.936170
1,positive,no,5.177778
2,negative,yes,4.394737
3,negative,no,4.475000


In [67]:
scenario_dict_outcome = {
    "1": {"positive": "1_and_4", "negative": "2_and_3"},
    "2": {"positive": "2_and_3", "negative": "1_and_4"},
    "3": {"positive": "2_and_3", "negative": "1_and_4"},
    "4": {"positive": "1_and_4", "negative": "2_and_3"},
}

scenario_dict_pa = {
    "1": {"yes": "1_and_4", "no": "2_and_3"},
    "2": {"yes": "2_and_3", "no": "1_and_4"},
    "3": {"yes": "2_and_3", "no": "1_and_4"},
    "4": {"yes": "1_and_4", "no": "2_and_3"},
}


def get_mean_scales(responsibility_scale, scenario_nr, solution_type, outcome, pa):
    scale_string = ""
    if responsibility_scale == "feel":
        scale_string += "FR"
    else:
        scale_string += "AR"

    scale_string += f"S{scenario_nr}"

    if solution_type == "familiar":
        scale_string += "T1"
    else:
        scale_string += "T2"

    outcome_filter = scenario_dict_outcome[str(scenario_nr)][outcome]
    pa_filter = scenario_dict_pa[str(scenario_nr)][pa]

    return data_all.loc[
        (data_all.positive_szenarios == outcome_filter)
        & (data_all.treatment_pa == pa_filter),
        scale_string,
    ].mean()

In [59]:
data_all.head()

,positive_szenarios,treatment_pa,PRS1T1,PRS1T2,PRS2T1,PRS2T2,PRS3T1,PRS3T2,PRS4T1,PRS4T2,...,ETHS4T2,PROS1T1,PROS1T2,PROS2T1,PROS2T2,PROS3T1,PROS3T2,PROS4T1,PROS4T2,origin
0,1_and_4,1_and_4,1.0,6.0,2.0,6.0,3.0,3.0,5.0,4.0,...,5.0,7.0,6.0,7.0,3.0,5.0,3.0,3.0,5.0,USA
1,2_and_3,1_and_4,1.0,6.0,1.0,6.0,1.0,6.0,2.0,6.0,...,3.0,5.0,4.0,7.0,7.0,6.0,6.0,6.0,3.0,USA
2,1_and_4,2_and_3,1.0,7.0,4.0,6.0,1.0,7.0,7.0,1.0,...,7.0,6.0,6.0,5.0,6.0,6.0,4.0,1.0,7.0,USA
4,1_and_4,1_and_4,5.0,4.0,2.0,7.0,2.0,7.0,4.0,5.0,...,6.0,7.0,7.0,2.0,3.0,2.0,2.0,6.0,6.0,USA
5,2_and_3,1_and_4,3.0,4.0,1.0,7.0,4.0,4.0,6.0,1.0,...,7.0,5.0,4.0,7.0,3.0,6.0,5.0,3.0,7.0,USA


In [63]:
# Feel Responsible + Scenario 1 + Familiar solution + positive outcome + PA

data_all.loc[
    (data_all.positive_szenarios == "1_and_4") & (data_all.treatment_pa == "1_and_4"), "FRS1T1"
].mean()

4.9361702127659575

In [108]:
import plotly.express as px

print(px.colors.sequential.Plasma)

['#0d0887', '#46039f', '#7201a8', '#9c179e', '#bd3786', '#d8576b', '#ed7953', '#fb9f3a', '#fdca26', '#f0f921']


In [116]:
print(px.colors.qualitative.D3)

['#1F77B4', '#FF7F0E', '#2CA02C', '#D62728', '#9467BD', '#8C564B', '#E377C2', '#7F7F7F', '#BCBD22', '#17BECF']


In [ ]:
"#1F77B4"
"#FF7F0E"
"#2CA02C"
"#D62728"
"#9467BD"
"#8C564B"
"#E377C2"
"#7F7F7F"

In [119]:
x_axis_values_list = [
    ["Familiar solution", "New solution"],
    ["Follow advise", "Not follow advise"],
    ["Hold on", "Change decision"],
    ["Wait and see", "Take action"],
]

colors_dict = {
    "feel responsible + positive outcome + PA yes": "#1F77B4",
    "feel responsible + positive outcome + PA no": "#FF7F0E",
    "feel responsible + negative outcome + PA yes": "#2CA02C",
    "feel responsible + negative outcome + PA no": "#D62728",
    "act responsible + positive outcome + PA yes": "#9467BD",
    "act responsible + positive outcome + PA no": "#8C564B",
    "act responsible + negative outcome + PA yes": "#E377C2",
    "act responsible + negative outcome + PA no": "#7F7F7F",
}

from plotly.subplots import make_subplots


fig = make_subplots(rows=2, cols=2)

# Create traces
for scenario, x_axis_values in zip(range(1, 5), x_axis_values_list):
    for responsibility_scale in ["feel", "act"]:
        for outcome in ["positive", "negative"]:
            for pa_available in ["yes", "no"]:
                fig.add_trace(
                    go.Scatter(
                        x=x_axis_values,
                        y=[
                            get_mean_scales(
                                responsibility_scale=responsibility_scale,
                                scenario_nr=scenario,
                                solution_type="familiar",
                                outcome=outcome,
                                pa=pa_available,
                            ),
                            get_mean_scales(
                                responsibility_scale=responsibility_scale,
                                scenario_nr=scenario,
                                solution_type="new",
                                outcome=outcome,
                                pa=pa_available,
                            ),
                        ],
                        mode="lines+markers",
                        name=f"{responsibility_scale} responsible + {outcome} outcome + PA {pa_available}",
                        legendgroup=f"{responsibility_scale} responsible + {outcome} outcome + PA {pa_available}",
                        marker_color=colors_dict[
                            f"{responsibility_scale} responsible + {outcome} outcome + PA {pa_available}"
                        ],
                        showlegend=True if scenario == 1 else False,
                    ),
                    row=1 if scenario in [1, 2] else 2,
                    col=1 if scenario in [1, 3] else 2,
                )
# fig.update_xaxes(range=[1, 7], dtick=1)
fig.update_yaxes(range=[1, 7], dtick=1)
fig.update_layout(width=900 + 200, height=600 + 200, title=f"Scenarios")
fig.show()
fig.write_html(f"scenarios_all.html")